In [12]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import precision_recall_fscore_support, classification_report, confusion_matrix
from scipy.stats import kurtosis, skew, pearsonr
from skimage.color import rgb2gray

In [20]:

def get_kurtosis(image_path):
    # 1. Read image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    kurtosis_value = kurtosis(img, axis=None)
    return kurtosis_value


In [3]:
def calculate_ycbcr_chrominance_kurtosis(image_path):
    """
    Calculates the Fisher kurtosis of the Cb and Cr chrominance channels 
    of an image.
    
    Args:
        image_path (str): The file path to the image.
        
    Returns:
        tuple: (kurtosis_cb, kurtosis_cr) or (None, None) if image fails to load.
    """
    # 1. Read the image
    # cv2.imread loads the image in BGR format by default
    img_bgr = cv2.imread(image_path)
    
    if img_bgr is None:
        print(f"Error: Could not load image at {image_path}")
        return None, None

    # 2. Convert the image to YCrCb (OpenCV uses YCrCb, not YCbCr)
    # The channels in the resulting array will be Y, Cr, Cb.
    # Note: OpenCV's convention is BGR and YCrCb.
    img_ycrcb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YCrCb)
    
    # 3. Extract the Cr and Cb channels
    # Channel 0 is Y (Luminance)
    # Channel 1 is Cr (Red-difference chrominance)
    # Channel 2 is Cb (Blue-difference chrominance)
    
    # We are interested in Cb (index 2) and Cr (index 1)
    Y, cr_channel, cb_channel = cv2.split(img_ycrcb)
    
    # Reshape the 2D channel arrays into 1D for kurtosis calculation
    cb_data = cb_channel.ravel()
    cr_data = cr_channel.ravel()
    
    # Calculate Fisher's Excess Kurtosis (default for scipy.stats.kurtosis is 
    # fisher=True, axis=0, which is correct here since we used ravel())
    # Fisher kurtosis (Excess Kurtosis) is 0 for a normal distribution.
    kurtosis_cb = kurtosis(cb_data)
    kurtosis_cr = kurtosis(cr_data)
    
    return kurtosis_cb, kurtosis_cr

In [4]:
def calculate_laplacian_residual_kurtosis(image_path):
    """
    Calculates the Fisher kurtosis of the Cb and Cr chrominance channel residuals 
    after applying a Laplacian filter (which acts as a high-pass filter).
    
    Args:
        image_path (str): The file path to the image.
        
    Returns:
        tuple: (kurtosis_cb_residual, kurtosis_cr_residual) or (None, None) 
               if the image fails to load.
    """
    # 1. Read the image
    img_bgr = cv2.imread(image_path)
    
    if img_bgr is None:
        print(f"Error: Could not load image at {image_path}")
        return None, None

    # 2. Convert to YCrCb (OpenCV's convention: Y, Cr, Cb)
    img_ycrcb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YCrCb)
    
    # Extract the Cb and Cr channels
    # Channel 1 is Cr (Red-difference chrominance)
    # Channel 2 is Cb (Blue-difference chrominance)
    cr_channel = img_ycrcb[:, :, 1]
    cb_channel = img_ycrcb[:, :, 2]
    
    # 3. Apply the Laplacian filter to each chrominance channel
    # cv2.Laplacian computes the second spatial derivative. 
    # ddepth=-1 means the output image will have the same depth (data type) as the source.
    # We use a kernel size of ksize=3 (the default)
    
    # NOTE: The Laplacian output (the residual) can contain negative values, 
    # so we'll use np.float64 to prevent clipping and maintain data integrity.
    # The 'residual' image here is the high-frequency component (noise/texture).

    # Convert to float and then apply Laplacian
    # cr_float = cr_channel.astype(np.float64)
    # cb_float = cb_channel.astype(np.float64)
    
    # Calculate the residual (filtered image)
    cr_residual = cv2.Laplacian(cr_channel, ddepth=cv2.CV_64F, ksize=3)
    cb_residual = cv2.Laplacian(cb_channel, ddepth=cv2.CV_64F, ksize=3)
    
    # 4. Compute Kurtosis of the Residuals
    
    # Flatten the 2D residual arrays into 1D for kurtosis calculation
    cr_data = cr_residual.ravel()
    cb_data = cb_residual.ravel()
    
    # Calculate Fisher's Excess Kurtosis. 
    # Kurtosis of noise is often high (leptokurtic) due to heavy tails.
    kurtosis_cr_residual = kurtosis(cr_data)
    kurtosis_cb_residual = kurtosis(cb_data)
    
    return kurtosis_cb_residual, kurtosis_cr_residual

In [5]:
import cv2
import numpy as np

def calculate_rgb_inter_channel_correlation(image_path):
    """
    Calculates the Pearson correlation coefficients between the R, G, and B 
    channels of an image.
    
    Args:
        image_path (str): The file path to the image.
        
    Returns:
        dict: A dictionary containing the correlation coefficients for R-G, R-B, 
              and G-B pairs, or None if the image fails to load.
    """
    # 1. Read the image
    # OpenCV reads images in BGR format by default
    img_bgr = cv2.imread(image_path)
    
    if img_bgr is None:
        print(f"Error: Could not load image at {image_path}")
        return None

    # 2. Split and Flatten the Channels
    # Split the image into its B, G, R components
    b_channel, g_channel, r_channel = cv2.split(img_bgr)
    
    # Flatten the 2D arrays (H x W) into 1D vectors (H*W)
    # This prepares the data for standard statistical correlation
    r_flat = r_channel.ravel()
    g_flat = g_channel.ravel()
    b_flat = b_channel.ravel()
    
    # plt.hist(r_channel.flatten(), bins=25, color='red')
    # plt.title('Histogram - Red')
    # plt.grid()
    # plt.show()

    # plt.hist(g_channel.flatten(), bins=25, color='green')
    # plt.title('Histogram - green')
    # plt.grid()
    # plt.show()


    # plt.hist(b_channel.flatten(), bins=25, color='blue')
    # plt.title('Histogram - blue')
    # plt.grid()
    # plt.show()
    # 3. Combine the flattened channels into a single array for np.corrcoef
    # The array should have shape (3, N), where N is the total number of pixels.
    data_matrix = np.stack([r_flat, g_flat, b_flat], axis=0)

    # 4. Compute the Pearson Correlation Coefficient Matrix
    # np.corrcoef returns an N x N matrix, where N is the number of rows in the input array.
    # The matrix M will be:
    # [[ corr(R,R), corr(R,G), corr(R,B) ],
    #  [ corr(G,R), corr(G,G), corr(G,B) ],
    #  [ corr(B,R), corr(B,G), corr(B,B) ]]
    correlation_matrix = np.corrcoef(data_matrix)
    correlation_coefficient, p_value = pearsonr(r_flat, g_flat)
    # 5. Extract the relevant off-diagonal correlation coefficients
    # (Since corr(X,Y) = corr(Y,X) and corr(X,X) = 1)
    
    r_g_corr = correlation_matrix[0, 1]  # Correlation between R and G
    r_b_corr = correlation_matrix[0, 2]  # Correlation between R and B
    g_b_corr = correlation_matrix[1, 2]  # Correlation between G and B

    return r_g_corr, r_b_corr, g_b_corr


In [6]:
#https://gemini.google.com/u/0/app/5474508fb11c1d95
def calculate_rgb_kurtosis_and_sckew(image_path):
    """
    Calculates the Pearson correlation coefficients between the R, G, and B 
    channels of an image.
    
    Args:
        image_path (str): The file path to the image.
        
    Returns:
        dict: A dictionary containing the correlation coefficients for R-G, R-B, 
              and G-B pairs, or None if the image fails to load.
    """
    # 1. Read the image
    # OpenCV reads images in BGR format by default
    img_bgr = cv2.imread(image_path)
    b_channel, g_channel, r_channel = cv2.split(img_bgr)
        
    r_flat = r_channel.ravel()
    g_flat = g_channel.ravel()
    b_flat = b_channel.ravel()
    
    r_kurtosis = kurtosis(r_flat) 
    g_kurtosis = kurtosis(g_flat)
    b_kurtosis = kurtosis(b_flat)

    r_skew = skew(r_flat)
    g_skew = skew(g_flat)
    b_skew = skew(b_flat)

    return r_kurtosis, g_kurtosis, b_kurtosis, r_skew, g_skew, b_skew

In [7]:
# https://python.plainenglish.io/introduction-to-image-processing-with-python-18fec2d8dff8
def spectral_flatness(spectrum: np.ndarray):
    def geometric_mean(x):
        if np.any(x == 0):
            return 0.0
        return np.exp(np.mean(np.log(x)))

    ps = np.abs(spectrum) ** 2
    return geometric_mean(ps) / np.mean(ps)

In [18]:
def get_spectrum(image_path):
    image = cv2.imread(image_path)
    
    gray_image = rgb2gray(image)
    f_image = np.fft.fft2(gray_image)
    
    fshift = np.fft.fftshift(f_image)
    
    magnitude_spectrum = np.log(np.abs(fshift))
    spectrum = spectral_flatness(magnitude_spectrum)
    return spectrum
    # plt.imshow(magnitude_spectrum, cmap='gray')
    # plt.title('Magnitude Spectrum')
    # plt.axis('off')
    # plt.show()
    # 

In [9]:

def two_simple_features(image_path):
    """
    Returns (mean_intensity, edge_density) for a single image.
    - mean_intensity: average grayscale intensity [0..255]
    - edge_density: proportion of edge pixels using Sobel magnitude threshold
    """
    # Load as RGB then convert to grayscale (uint8)
    img = Image.open(image_path).convert("RGB")
    gray = np.array(img.convert("L"))  # shape (H, W), dtype uint8

    # Feature 1: mean intensity
    mean_intensity = float(gray.mean())

    # Feature 2: edge density using Sobel magnitude
    # Use OpenCV Sobel on float32 normalized [0..1] to get good gradient scale
    gray_f = gray.astype(np.float32) / 255.0
    gx = cv2.Sobel(gray_f, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=3)
    gy = cv2.Sobel(gray_f, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=3)
    mag = np.sqrt(gx**2 + gy**2)

    # Simple threshold at 0.2 (tune as you like)
    thresh = 0.2
    edges = (mag > thresh).astype(np.uint8)
    edge_density = float(edges.mean())  # fraction of edge pixels

    return mean_intensity, edge_density


In [21]:
rows = []
for folder, label in [("imagenet_midjourney/test/ai", 1), ("imagenet_midjourney/test/nature", 0)]:
    for fname in os.listdir(folder):
        path = os.path.join(folder, fname)
        if not os.path.isfile(path): 
            continue
        f1, f2 = two_simple_features(path)
        kurtosis_value = get_kurtosis(path)
        kurtosis_cb, kurtosis_cr = calculate_ycbcr_chrominance_kurtosis(path)
        kurtosis_cb_residual, kurtosis_cr_residual = calculate_laplacian_residual_kurtosis(path)
        r_g_corr, r_b_corr, g_b_corr = calculate_rgb_inter_channel_correlation(path)
        r_kurtosis, g_kurtosis, b_kurtosis, r_skew, g_skew, b_skew = calculate_rgb_kurtosis_and_sckew(path)
        spectrum = get_spectrum(path)
        
        rows.append({
            "file_path": path,
            "mean_intensity": f1,
            "edge_density": f2,
            "kurtosis": kurtosis_value,
            "kurtosis_cb" : kurtosis_cb,
            "kurtosis_cr" : kurtosis_cr,
            "kurtosis_cb_residual": kurtosis_cb_residual,
            "kurtosis_cr_residual": kurtosis_cr_residual,
            "r_g_corr": r_g_corr,
            "r_b_corr": r_b_corr,
            "g_b_corr": g_b_corr,
            "r_kurtosis": r_kurtosis,
            "g_kurtosis": g_kurtosis,
            "b_kurtosis": b_kurtosis,
            "r_skew": r_skew,
            "g_skew": g_skew,
            "b_skew": b_skew,
            "spectrum": spectrum,
            "label": label
                
        })

df = pd.DataFrame(rows)
display(df)

/var/folders/c5/kvs4zf7x2sl0k1z2fdtkxl5h0000gn/T/ipykernel_2935/422008611.py:40: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis_cb = kurtosis(cb_data)
/var/folders/c5/kvs4zf7x2sl0k1z2fdtkxl5h0000gn/T/ipykernel_2935/422008611.py:41: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis_cr = kurtosis(cr_data)


,file_path,mean_intensity,edge_density,kurtosis,kurtosis_cb,kurtosis_cr,kurtosis_cb_residual,kurtosis_cr_residual,r_g_corr,r_b_corr,g_b_corr,r_kurtosis,g_kurtosis,b_kurtosis,r_skew,g_skew,b_skew,spectrum,label
0,imagenet_midjourney/test/ai/889_midjourney_88.png,117.090787,0.161981,-0.889811,-0.607025,0.148531,2.793006,2.531067,0.842543,0.607587,0.917658,-1.127755,-0.956213,-0.834478,-0.000841,0.340065,0.583515,0.333780,1
1,imagenet_midjourney/test/ai/593_midjourney_88.png,66.074798,0.146701,0.059336,5.700131,-0.495998,8.801154,0.455845,0.999961,0.999954,0.999940,0.044632,0.062334,0.032893,0.943361,0.951317,0.939376,0.825935,1
2,imagenet_midjourney/test/ai/721_midjourney_169...,170.313185,0.022654,-0.561404,-0.068263,-0.228240,2.956710,2.324420,0.971984,0.919173,0.983215,-0.589852,-0.499418,-0.933069,-0.426347,-0.541021,-0.395986,0.359850,1
3,imagenet_midjourney/test/ai/110_midjourney_88.png,89.795885,0.213519,-0.179947,0.413666,-0.701424,0.913915,0.607857,0.813153,0.788791,0.814246,-0.676220,-0.002694,1.064305,0.253491,0.502331,1.132086,0.225965,1
4,imagenet_midjourney/test/ai/811_midjourney_169...,109.457084,0.291970,-1.018108,1.076538,3.380314,38.354430,22.135910,0.964528,0.877581,0.966933,-1.192001,-0.983584,-0.647029,0.113744,0.330212,0.638316,0.602762,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,imagenet_midjourney/test/nature/ILSVRC2012_val...,186.963665,0.096384,0.747218,0.444295,-1.118251,14.799665,15.736582,0.998407,0.998964,0.999497,0.956857,0.625452,0.866677,-1.446542,-1.312883,-1.384354,0.625383,0
996,imagenet_midjourney/test/nature/ILSVRC2012_val...,121.042464,0.214816,-0.966696,-0.399458,-0.843093,0.965760,1.535850,0.886512,0.845178,0.922395,-0.231864,-0.877756,-1.378061,-0.493278,-0.103903,-0.129693,0.623614,0
997,imagenet_midjourney/test/nature/ILSVRC2012_val...,124.374005,0.228579,-1.216200,6.336707,10.313181,69.706880,103.394087,0.987688,0.947262,0.972914,-1.247139,-1.216559,-1.092762,-0.105638,0.006985,-0.095163,0.490304,0
998,imagenet_midjourney/test/nature/ILSVRC2012_val...,157.713980,0.241051,-1.010681,1.749331,3.788984,7.584777,7.907482,0.926960,0.903440,0.976565,-0.549031,-1.102855,-1.170849,-0.923623,-0.725973,-0.733164,0.665653,0


In [23]:
X = df.drop(['label', 'file_path'], axis=1) # Features
y = df['label'] # Target

In [24]:
X

,mean_intensity,edge_density,kurtosis,kurtosis_cb,kurtosis_cr,kurtosis_cb_residual,kurtosis_cr_residual,r_g_corr,r_b_corr,g_b_corr,r_kurtosis,g_kurtosis,b_kurtosis,r_skew,g_skew,b_skew,spectrum
0,117.090787,0.161981,-0.889811,-0.607025,0.148531,2.793006,2.531067,0.842543,0.607587,0.917658,-1.127755,-0.956213,-0.834478,-0.000841,0.340065,0.583515,0.333780
1,66.074798,0.146701,0.059336,5.700131,-0.495998,8.801154,0.455845,0.999961,0.999954,0.999940,0.044632,0.062334,0.032893,0.943361,0.951317,0.939376,0.825935
2,170.313185,0.022654,-0.561404,-0.068263,-0.228240,2.956710,2.324420,0.971984,0.919173,0.983215,-0.589852,-0.499418,-0.933069,-0.426347,-0.541021,-0.395986,0.359850
3,89.795885,0.213519,-0.179947,0.413666,-0.701424,0.913915,0.607857,0.813153,0.788791,0.814246,-0.676220,-0.002694,1.064305,0.253491,0.502331,1.132086,0.225965
4,109.457084,0.291970,-1.018108,1.076538,3.380314,38.354430,22.135910,0.964528,0.877581,0.966933,-1.192001,-0.983584,-0.647029,0.113744,0.330212,0.638316,0.602762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,186.963665,0.096384,0.747218,0.444295,-1.118251,14.799665,15.736582,0.998407,0.998964,0.999497,0.956857,0.625452,0.866677,-1.446542,-1.312883,-1.384354,0.625383
996,121.042464,0.214816,-0.966696,-0.399458,-0.843093,0.965760,1.535850,0.886512,0.845178,0.922395,-0.231864,-0.877756,-1.378061,-0.493278,-0.103903,-0.129693,0.623614
997,124.374005,0.228579,-1.216200,6.336707,10.313181,69.706880,103.394087,0.987688,0.947262,0.972914,-1.247139,-1.216559,-1.092762,-0.105638,0.006985,-0.095163,0.490304
998,157.713980,0.241051,-1.010681,1.749331,3.788984,7.584777,7.907482,0.926960,0.903440,0.976565,-0.549031,-1.102855,-1.170849,-0.923623,-0.725973,-0.733164,0.665653


In [25]:
y.head

<bound method NDFrame.head of 0      1
1      1
2      1
3      1
4      1
      ..
995    0
996    0
997    0
998    0
999    0
Name: label, Length: 1000, dtype: int64>

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [49]:
!uv pip install xgboost

Using Python 3.13.2 environment at: venv
Resolved 3 packages in 389ms                                         
Prepared 1 package in 221ms                                              
Installed 1 package in 4ms                                  
 + xgboost==3.1.1


In [29]:

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [30]:
y_pred = rf.predict(X_test)




In [36]:
y_pred

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1])

In [32]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.875


In [51]:
import numpy as np

from scipy.stats import uniform, randint

from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb

In [54]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric="auc", early_stopping_rounds=5)


xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)])

y_pred = xgb_model.predict(X_test)

accuracy_score(y_test, y_pred)

[0]	validation_0-auc:0.93715
[1]	validation_0-auc:0.95052
[2]	validation_0-auc:0.95643
[3]	validation_0-auc:0.95508
[4]	validation_0-auc:0.95172
[5]	validation_0-auc:0.95092
[6]	validation_0-auc:0.95363
[7]	validation_0-auc:0.95247


0.9

In [57]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:

# ---- Create a random binary dataset (demo) ----
np.random.seed(42)
n_samples = 300
n_features = 8
X = np.random.randn(n_samples, n_features)      # random features
y = np.random.randint(0, 2, size=n_samples)     # random labels 0/1

# ---- 10-fold CV with cross_val_predict ----
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
clf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42, n_jobs=-1)

# cross_val_predict returns out-of-fold predictions for every sample
y_pred = cross_val_predict(clf, X, y, cv=cv)

prec, rec, f1, _ = precision_recall_fscore_support(y, y_pred, average="binary")
print(f"Precision: {prec:.3f}  Recall: {rec:.3f}  F1: {f1:.3f}")

print("\nClassification report:\n", classification_report(y, y_pred, digits=3))
print("\nConfusion matrix:\n", confusion_matrix(y, y_pred))
